In [1]:
import biblio_modules

from datetime import datetime
import pyarrow as py
import pandas as pd
import numpy as np
import time
import pytz

In [ ]:
transactions_processing = pd.read_csv("../Source/Transactions.csv", sep=";")

In [ ]:

date_time_format = "2021-03-01 24:01:07.843138"

date_time_obj = datetime.strptime(date_time_format, "%Y-%m-%d %H:%M:%S.%f")

unix_timestamp = int(date_time_obj.timestamp())

unix_timestamp


In [ ]:
date_h_ete_2021 = transactions_processing.loc[transactions_processing['date'].str.contains('2021-03-28')]
date_h_ete_2021
#L'heure à avancé d'une heure, passant de 2h à 3h, ce qui à décalé le temps en 25h (absence de la 2ème heure)
date_h_hiver_2022 = transactions_processing.loc[transactions_processing['date'].str.contains('2021-10-31')]
date_h_hiver_2022
date_h_ete_2022 = transactions_processing.loc[transactions_processing['date'].str.contains('2022-03-27')]
date_h_ete_2022
date_h_hiver_2023 = transactions_processing.loc[transactions_processing['date'].str.contains('2022-03-27')]
date_h_hiver_2023